Author: Jonathan Markel

Notes: 
- unsure if pressure transducer measurements are accurate 
- numbers are all too high, making me think there is a scaling error somewhere in the given values or in the experimental data


# Part 1: Polycarbonate Cylinder

In [1]:
import pandas as pd

In [2]:
GF = 2.11 # gage factor +- 0.5%
Vin = 3.3 #V

In [3]:
# polycarbonate material properties
E_p = 354000 # Youngs Modulus (psi)
nu_p = 0.37 # Poissons ratio
r_p = 2 # radius (inches)
t_p = 0.125 # thickness (inches)

In [4]:
# Experimental data
data1 = pd.read_excel('data.xlsx', sheet_name='oil_RAW')
data1.columns = ['bourdon', 'bridge', 'transducer']
display (data1)

,bourdon,bridge,transducer
0,0.0,-8.89936,5.26127
1,5.0,-8.70373,5.27603
2,10.5,-8.52612,5.27932
3,15.0,-8.37288,5.28271
4,20.5,-8.21049,5.28546
5,25.1,-8.07250,5.28793
6,30.0,-7.90872,5.29109
7,35.0,-7.73974,5.29397
8,40.5,-7.56643,5.29735
9,46.0,-7.39728,5.30021


__Calculate pressure using hoop stress equation and compare with pressure transducer measurements.__

Theoretical hoop stress equation ($\epsilon_{h} = \frac{Pr}{Et}$)

In [5]:
strain_hs = data1['transducer']# - data1['transducer'].min() 
# Is the transducer supposed to be normalized to 0?
strain_hs

0     5.26127
1     5.27603
2     5.27932
3     5.28271
4     5.28546
5     5.28793
6     5.29109
7     5.29397
8     5.29735
9     5.30021
10    5.30276
Name: transducer, dtype: float64

In [6]:
P_hs = strain_hs * E_p * t_p / r_p
P_hs
# Unreasonable numbers

0     116405.59875
1     116732.16375
2     116804.95500
3     116879.95875
4     116940.80250
5     116995.45125
6     117065.36625
7     117129.08625
8     117203.86875
9     117267.14625
10    117323.56500
Name: transducer, dtype: float64

In [7]:
p1 = data1['bourdon']
strain_hs0 = p1 * r_p / (E_p * t_p)
strain_hs0
# does not match with normalized transducer

0     0.000000
1     0.000226
2     0.000475
3     0.000678
4     0.000927
5     0.001134
6     0.001356
7     0.001582
8     0.001831
9     0.002079
10    0.002260
Name: bourdon, dtype: float64

Full bridge strain $ \epsilon = \frac{-2V_r}{G[(\nu+1)-V_r(\nu-1)]}$

In [8]:
#Checking to see how bridge strain compares with measured strain
Vout1 = data1['bridge']
Vr1 = Vout1/Vin

strain_b = (-2 * Vr1) / ( GF * ( (nu_p + 1) - Vr1 * (nu_p-1) ) )
strain_b
# No way these numbers are right

0      -7.770300
1      -8.572746
2      -9.502707
3     -10.526897
4     -11.943449
5     -13.550553
6     -16.243816
7     -20.663376
8     -29.172037
9     -50.339730
10   -120.205112
Name: bridge, dtype: float64

# Part 2 - Aluminum Soda Can

In [9]:
E_a = 1*10**7 # psi
nu_a = 0.33
r_a = 1.302 #inches
t_a = 0.0035 # inches

In [10]:
data2 = pd.read_excel('data.xlsx', sheet_name='soda', header=None, index_col=0)
data2.columns = ['bridge']
display(data2)

,bridge
0,
before,0.071624
after,-0.533438


__Calculate the resulting strain from opening the soda can. Then use this strain to calculate the pressure in the unopened can.__

Quarter bridge strain $ \epsilon = \frac{-3 V_r}{G(1+2V_r} $, where $V_r = V_{out}/V_{in}$

Hoop strain $\epsilon = \frac{Pr}{Et} (1-\nu/2)$

In [11]:
data2 = data2 - data2.values[0]
data2

,bridge
0,
before,0.000000
after,-0.605062


In [12]:
# Calculating resulting strain after opening the can
Vout_after = data2.loc['after'].values[0]
Vr2 = Vout_after/Vin

strain_after = (-3 * Vr2) / ( GF * (1 + 2 * Vr2)) # using 3 per Marc's message

strain_after

0.4116406665403648

Pressure of a room temperature soda can should be around 50-60 psi.

In [13]:
# Rearranging hoop strain for thin walled cylindrical can to solve for pressure
p2 = (strain_after * E_a * t_a) / (r_a * (1 - nu_a / 2))

p2 # in psi
# 13000 psi is way too high

13252.226725270904